# Imports

In [ ]:
!pip install yfinance --upgrade --no-cache-dir
!pip install finnhub-python
!pip3 install news-please

!pip install torch
!pip install peft
!pip install -U accelerate
!pip install transformers bitsandbytes
!pip install pynvml
!pip install gnews

Rank the sentences from an article

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict
import heapq

nltk.download('punkt')
nltk.download('stopwords')

def rank_sentences(text):
    stop_words = set(stopwords.words('english'))
    word_frequencies = defaultdict(int)
    
    # Tokenize words and count frequencies, ignoring stopwords
    for word in word_tokenize(text):
        if word.lower() not in stop_words:
            word_frequencies[word.lower()] += 1
    
    # Find the maximum frequency
    max_frequency = max(word_frequencies.values())
    
    # Normalize frequencies
    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word] / max_frequency)
    
    sentence_scores = defaultdict(int)
    sentences = sent_tokenize(text)
    
    # Score sentences by summing normalized frequencies of their words
    for sent in sentences:
        for word in word_tokenize(sent.lower()):
            if word in word_frequencies:
                sentence_scores[sent] += word_frequencies[word]
    
    # Get top n sentences based on their scores
    summary_sentences = heapq.nlargest(3, sentence_scores, key=sentence_scores.get)
    return summary_sentences

# Example usage
# key_sentences = rank_sentences(article.text)
# for sentence in key_sentences:
#     print(sentence)

[nltk_data] Downloading package punkt to C:\Users\bow33/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bow33/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


News from other sources (Google)

In [18]:
# !pip3 install newspaper3k
from gnews import GNews
import json
from datetime import datetime
search_terms = "Tesla stock"

google_news = GNews()
def get_news(search_terms):
    stock_news = google_news.get_news(search_terms)
    # Define a function to convert date strings into datetime objects
    def parse_date(date_str):
        # Example date: "Thu, 08 Feb 2024 15:35:00 GMT"
        return datetime.strptime(date_str, '%a, %d %b %Y %H:%M:%S GMT')

    # Sort stock_news by parsed 'publish date'
    stock_news = sorted(stock_news, key=lambda x: parse_date(x['published_date']))
    
    print(stock_news[:4])
    return stock_news

def get_content(url):
    article = google_news.get_full_article(url)
    return article

def write_articles_to_file(tweets, file_path):
    """Generic function to write tweets to a specified file with deduplication based on title and published date."""
    new_tweets_count = 0
    existing_tweets = set()
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                tweet = json.loads(line)
                # Assuming 'content' as the title and 'published_date' as the date attributes
                key = (tweet['title'], tweet['published date'])
                existing_tweets.add(key)
    except FileNotFoundError:
        pass  # It's okay if the file does not exist yet

    with open(file_path, 'a', encoding='utf-8') as file:
        for tweet in tweets:
            # Again, assuming 'content' and 'published_date' are the relevant attributes
            key = (tweet['title'], tweet['published date'])
            if key not in existing_tweets:
                tweet['content'] = get_content(tweet['url'])
                file.write(json.dumps(tweet, ensure_ascii=False) + '\n')
                new_tweets_count += 1
                existing_tweets.add(key)

    return new_tweets_count

write_articles_to_file(stock_news, "tesla_stock.jsonl")

# key_sentences = rank_sentences(article.text)
# for sentence in key_sentences:
#     print(sentence)


0

In [4]:
print(stock_news)

[{'title': "Here's how much Tesla stock is down since Cybertruck launch - Finbold - Finance in Bold", 'description': "Here's how much Tesla stock is down since Cybertruck launch  Finbold - Finance in Bold", 'published date': 'Fri, 09 Feb 2024 14:37:46 GMT', 'url': 'https://news.google.com/rss/articles/CBMiT2h0dHBzOi8vZmluYm9sZC5jb20vaGVyZXMtaG93LW11Y2gtdGVzbGEtc3RvY2staXMtZG93bi1zaW5jZS1jeWJlcnRydWNrLWxhdW5jaC_SAQA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://finbold.com', 'title': 'Finbold - Finance in Bold'}}, {'title': "There Is A Black Cloud Around Tesla's Stock - Jalopnik", 'description': "There Is A Black Cloud Around Tesla's Stock  Jalopnik", 'published date': 'Thu, 08 Feb 2024 15:35:00 GMT', 'url': 'https://news.google.com/rss/articles/CBMiSmh0dHBzOi8vamFsb3BuaWsuY29tL3RoZXJlLWlzLWEtYmxhY2stY2xvdWQtYXJvdW5kLXRlc2xhcy1zdG9jay0xODUxMjM4NjM40gEA?oc=5&hl=en-GB&gl=GB&ceid=GB:en', 'publisher': {'href': 'https://jalopnik.com', 'title': 'Jalopnik'}}, {'title': 'Tesla

In [ ]:
# import yfinance as yf
# import finnhub
# finnhub_client = finnhub.Client(api_key="cmr6gghr01qvmr5q06u0cmr6gghr01qvmr5q06ug")

# msft = yf.Ticker("MSFT")
# msft.news
# # Keeping only 'title' and 'link' in each item
# msft_news_filtered = [{"title": item["title"], "link": item["link"]} for item in msft.news]

# print(msft_news_filtered)

**FinForcaster App.py**


In [ ]:
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

# print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [2]:
import os
import re
import time
import json
import random
import finnhub
import torch
# import gradio as gr
import pandas as pd
import yfinance as yf
from pynvml import *
from peft import PeftModel
from collections import defaultdict
from datetime import date, datetime, timedelta
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

finnhub_client = finnhub.Client(api_key="cmr6gghr01qvmr5q06u0cmr6gghr01qvmr5q06ug")

def print_gpu_utilization():

    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def get_curday():

    return date.today().strftime("%Y-%m-%d")


def n_weeks_before(date_string, n):
    # date = datetime.strptime(date_string, "%Y-%m-%d") - timedelta(days=7*n)
    date = datetime.strptime(date_string, "%Y-%m-%d") - timedelta(days=n)
    return date.strftime("%Y-%m-%d")

def n_days_before(date_string, n):
    date = datetime.strptime(date_string, "%Y-%m-%d") - timedelta(days=n)
    return date.strftime("%Y-%m-%d")


def get_stock_data(stock_symbol, steps):

    stock_data = yf.download(stock_symbol, steps[0], steps[-1])
    if len(stock_data) == 0:
        raise gr.Error(f"Failed to download stock price data for symbol {stock_symbol} from yfinance!")

#     print(stock_data)

    dates, prices = [], []
    available_dates = stock_data.index.format()

    for date in steps[:-1]:
        for i in range(len(stock_data)):
            if available_dates[i] >= date:
                prices.append(stock_data['Close'][i])
                dates.append(datetime.strptime(available_dates[i], "%Y-%m-%d"))
                break

    dates.append(datetime.strptime(available_dates[-1], "%Y-%m-%d"))
    prices.append(stock_data['Close'][-1])

    return pd.DataFrame({
        "Start Date": dates[:-1], "End Date": dates[1:],
        "Start Price": prices[:-1], "End Price": prices[1:]
    })


def get_news(symbol, data, profile):
    
    news_list = []
    filtered_company_name = ' '.join(profile['name'].split()[:1])
    print(profile)

    for end_date, row in data.iterrows():
        start_date = row['Start Date'].strftime('%Y-%m-%d')
        end_date = row['End Date'].strftime('%Y-%m-%d')
#         print(symbol, ': ', start_date, ' - ', end_date)
        # time.sleep(1) # control qpm
        weekly_news = finnhub_client.company_news(symbol, _from=start_date, to=end_date)
        if len(weekly_news) == 0:
            raise gr.Error(f"No company news found for symbol {symbol} from finnhub!")
        weekly_news = [
            {
                "date": datetime.fromtimestamp(n['datetime']).strftime('%Y-%m-%d %H:%M'),
                "headline": n['headline'],
                "summary": n['summary'],
                "url": n['url'],
                "source": n['source']
            } 
            for n in weekly_news
            if "zacks.com" not in n['summary'].lower() and 
               (symbol.lower() in n['headline'].lower() or filtered_company_name.lower() in n['headline'].lower())
        ]
        weekly_news.sort(key=lambda x: x['date'])
        news_list.append(json.dumps(weekly_news))

    data['News'] = news_list

    # additions:
    total_news_count = sum(len(json.loads(weekly_news)) for weekly_news in news_list)
    print("Number of news items after filtering:", total_news_count) 

    # To save the news to a file, use:
    print_or_save_news(data['News'], f"{symbol}_news.txt")

    return data

def print_or_save_news(news_list, filename=None):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write("")
    for news_items_str in news_list:
        news_items = json.loads(news_items_str)  # Load the JSON string into a list of dictionaries
        for news in news_items:
            news_str = f"Date: {news['date']}\nHeadline: {news['headline']}\nSummary: {news['summary']}\nURL: {news['url']}\nSource: {news['source']}\n\n"
            if filename:
                with open(filename, 'a', encoding='utf-8') as file:
                    file.write(news_str)
            else:
                print(news_str)

def get_company_prompt(symbol, profile):

    # profile = finnhub_client.company_profile2(symbol=symbol)
    if not profile:
        raise gr.Error(f"Failed to find company profile for symbol {symbol} from finnhub!")

    company_template = "[Company Introduction]:\n\n{name} is a leading entity in the {finnhubIndustry} sector. Incorporated and publicly traded since {ipo}, the company has established its reputation as one of the key players in the market. As of today, {name} has a market capitalization of {marketCapitalization:.2f} in {currency}, with {shareOutstanding:.2f} shares outstanding." \
        "\n\n{name} operates primarily in the {country}, trading under the ticker {ticker} on the {exchange}. As a dominant force in the {finnhubIndustry} space, the company continues to innovate and drive progress within the industry."

    formatted_str = company_template.format(**profile)

    return formatted_str


def get_prompt_by_row(symbol, row):

    start_date = row['Start Date'] if isinstance(row['Start Date'], str) else row['Start Date'].strftime('%Y-%m-%d')
    end_date = row['End Date'] if isinstance(row['End Date'], str) else row['End Date'].strftime('%Y-%m-%d')
    term = 'increased' if row['End Price'] > row['Start Price'] else 'decreased'
    head = "From {} to {}, {}'s stock price {} from {:.2f} to {:.2f}. Company news during this period are listed below:\n\n".format(
        start_date, end_date, symbol, term, row['Start Price'], row['End Price'])

    news = json.loads(row["News"])
    news = ["[Headline]: {}\n[Summary]: {}\n".format(
        n['headline'], n['summary']) for n in news if n['date'][:8] <= end_date.replace('-', '') and \
        not n['summary'].startswith("Looking for stock market analysis and research with proves results?")]

    basics = json.loads(row['Basics'])
    if basics:
        basics = "Some recent basic financials of {}, reported at {}, are presented below:\n\n[Basic Financials]:\n\n".format(
            symbol, basics['period']) + "\n".join(f"{k}: {v}" for k, v in basics.items() if k != 'period')
    else:
        basics = "[Basic Financials]:\n\nNo basic financial reported."

    return head, news, basics


def sample_news(news, k=5):

    return [news[i] for i in sorted(random.sample(range(len(news)), k))]


def get_current_basics(symbol, curday):

    basic_financials = finnhub_client.company_basic_financials(symbol, 'all')
    if not basic_financials['series']:
        raise gr.Error(f"Failed to find basic financials for symbol {symbol} from finnhub!")

    final_basics, basic_list, basic_dict = [], [], defaultdict(dict)

    for metric, value_list in basic_financials['series']['quarterly'].items():
        for value in value_list:
            basic_dict[value['period']].update({metric: value['v']})

    for k, v in basic_dict.items():
        v.update({'period': k})
        basic_list.append(v)

    basic_list.sort(key=lambda x: x['period'])

    for basic in basic_list[::-1]:
        if basic['period'] <= curday:
            break

    return basic


def get_all_prompts_online(symbol, data, curday, profile, with_basics=True):

    company_prompt = get_company_prompt(symbol, profile)

    prev_rows = []

    for row_idx, row in data.iterrows():
        head, news, _ = get_prompt_by_row(symbol, row)
        prev_rows.append((head, news, None))

    prompt = ""
    for i in range(-len(prev_rows), 0):
        prompt += "\n" + prev_rows[i][0]
        sampled_news = sample_news(
            prev_rows[i][1],
            min(5, len(prev_rows[i][1]))
        )
        if sampled_news:
            prompt += "\n".join(sampled_news)
        else:
            prompt += "No relative news reported."

    period = "{} to {}".format(curday, n_weeks_before(curday, -1))

    if with_basics:
        basics = get_current_basics(symbol, curday)
        basics = "Some recent basic financials of {}, reported at {}, are presented below:\n\n[Basic Financials]:\n\n".format(
            symbol, basics['period']) + "\n".join(f"{k}: {v}" for k, v in basics.items() if k != 'period')
    else:
        basics = "[Basic Financials]:\n\nNo basic financial reported."

    info = company_prompt + '\n' + prompt + '\n' + basics
    prompt = info + f"\n\nBased on all the information before {curday}, let's first analyze the positive developments and potential concerns for {symbol}. Come up with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. " \
        f"Then make your prediction of the {symbol} stock price movement for next week ({period}). Provide a summary analysis to support your prediction."

    return info, prompt


def construct_prompt(ticker, curday, n_weeks, use_basics):

    profile = finnhub_client.company_profile2(symbol=ticker)

    try:
        steps = [n_weeks_before(curday, n) for n in range(n_weeks + 1)][::-1]
    except Exception:
        raise gr.Error(f"Invalid date {curday}!")

    data = get_stock_data(ticker, steps)
    data = get_news(ticker, data, profile)
    data['Basics'] = [json.dumps({})] * len(data)
    # print(data)

    info, prompt = get_all_prompts_online(ticker, data, curday, profile, use_basics)

    prompt = B_INST + B_SYS + SYSTEM_PROMPT + E_SYS + prompt + E_INST
    # print(prompt)

    return info, prompt


def predict(ticker, date, n_weeks, use_basics):

    print_gpu_utilization()

    info, prompt = construct_prompt(ticker, date, n_weeks, use_basics)

    # additions:
    # save prompt to a file
    with open("./prompt.txt", 'w', encoding='utf-8') as file:
        file.write(prompt)

    inputs = tokenizer(
        prompt, return_tensors='pt', padding=False
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    print("Inputs loaded onto devices.")

    res = model.generate(
        **inputs, max_length=4096, do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True, streamer=streamer
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    answer = re.sub(r'.*\[/INST\]\s*', '', output, flags=re.DOTALL)

    torch.cuda.empty_cache()

    return info, answer, output

/tmp/ipykernel_1309/859819138.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,
    offload_folder="offload/"
)
model = PeftModel.from_pretrained(
    base_model,
    'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora',
    offload_folder="offload/"
)
model = model.eval()

tokenizer = AutoTokenizer.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    )

streamer = TextStreamer(tokenizer)

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

SYSTEM_PROMPT = "You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. " \
    "Your answer format should be as follows:\n\n[Positive Developments]:\n1. ...\n\n[Potential Concerns]:\n1. ...\n\n[Prediction & Analysis]\nPrediction: ...\nAnalysis: ..."

In [ ]:
count = 0
def extract_and_append_prediction_to_file(date, offset, option, ticker):
    # Define the pattern to search for
    pattern = r"(Down by \d+-\d+%|Up by \d+-\d+%)"
    # Find all occurrences of the pattern
    matches = re.findall(pattern, answer)
    # Open the file in append mode and write the matches
    with open(f"{ticker}_from:{date}_past_{option}:{offset}.txt", 'a', encoding='utf-8') as file:
        for match in matches:
            file.write(match + '\n')
            print("-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+ prediction")

def extract_and_append_reasoning_to_file(output, date, offset, option, ticker):
    # Open the file in append mode and write the matches
    with open(f"{ticker}_reasoning_from:{date}_past_{option}:{offset}.txt", 'a', encoding='utf-8') as file:
        file.write(output + '\n')
        file.write('-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+\n')
        print("-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+ output")

# Run the process 10 times
for _ in range(10):
    ticker = "TSLA"
    date = "2024-01-24" # get_curday()
    offset = 1

    option = {
        "days": "days",
        "weeks": "weeks"
    }
    option = "days"

    text, answer, output = predict(ticker, date, offset, False)
    extract_and_append_prediction_to_file(date, offset, option, ticker)
    extract_and_append_reasoning_to_file(output, date, offset, option, ticker)
    count+=1
    print("-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+", count)

In [7]:
def read_and_print_file(filename):
    try:
        with open(filename, 'r') as file:
            print(f"Contents of {filename}:")
            for line in file:
                print(line.strip())  # .strip() removes any leading/trailing whitespace, including newline characters
    except FileNotFoundError:
        print(f"File {filename} not found.")

# Example usage
filename = "TSLA_from:2024-01-24_past_days:1.txt"
#           TSLA_from:2023-11-9_past_days:3.txt
read_and_print_file(filename)

Contents of TSLA_from:2024-01-24_past_days:1.txt:
Up by 5-7%
Up by 3-4%
Up by 3-4%
Down by 0-1%
Down by 2-3%
Down by 0-1%
Up by 3-4%
Up by 3-4%
Up by 5-7%
Up by 2-3%


Find key sentences

Tesla stock price analysis

Indeed, since November 30, 2023, the price of Tesla stock has declined from $240.07 to the present $189.56, which indicates a 21.04% drop over the last 10 weeks, albeit recording a slight recovery in the past week, during which its price increased by 2.57%, according to the data retrieved on February 9.
Source: Finbold

Meanwhile, Tesla has also issued a disappointing earnings report for the fourth quarter of 2023, which the company’s biggest bull, Wedbush Securities’ Daniel Ives, referred to as a ‘train wreck,’ and which has led investors to start second-guessing the company’s prospects as an automaker.
On top of that, the beaten-down stock is displaying negative signs in short, medium, and long-term trends, performing worse than 76% of all assets in the stock market in the last year, as well as trading in the lower part of both its 52-week range and its last-month range.


[nltk_data] Downloading package punkt to C:\Users\bow33/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bow33/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
